In [ ]:
import torch
from torchvision.io import read_image
from torchvision.utils import draw_segmentation_masks,draw_bounding_boxes
from torchvision.ops import masks_to_boxes
import torchvision.transforms.functional as F

import numpy as np
from path import Path
import matplotlib.pyplot as plt

plt.rcParams["savefig.bbox"] = "tight"

## 实例分割转目标检测

In [ ]:
def show(imgs):
    if not isinstance(imgs, list):
        imgs = [imgs]
    fix, axs = plt.subplots(ncols=len(imgs), squeeze=False)
    for i, img in enumerate(imgs):
        img = img.detach()
        img = F.to_pil_image(img)
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

In [ ]:
maskpath=Path(r'0.png')
imagepath=Path(r'0.jpg')
mask=read_image(maskpath)
img=read_image(imagepath)

obj_ids = torch.unique(mask)

# first id is the background, so remove it.
obj_ids = obj_ids[1:]

# split the color-encoded mask into a set of boolean masks.
# Note that this snippet would work as well if the masks were float values instead of ints.
masks = mask == obj_ids[:, None, None]
masks.size()

In [ ]:

drawn_masks = []
for mask in masks:
    drawn_masks.append(draw_segmentation_masks(img, mask, alpha=0.8, colors="blue"))

show(drawn_masks)

In [ ]:
boxes = masks_to_boxes(masks)
print(boxes.size())
print(boxes)

In [ ]:
drawn_boxes = draw_bounding_boxes(img, boxes, colors="red")
show(drawn_boxes)

## 语义分割转目标检测

In [ ]:
import cv2
from path import Path
from skimage import morphology,measure
from box import Box,Label,ImageDir
from functools import partial

In [ ]:
def getboundingbox(imdata,min_size=10):
    if isinstance(imdata,Label):
        imdata=cv2.imread(imdata.label_path,cv2.IMREAD_GRAYSCALE)
    mask=imdata>0
    # 删掉小于min_size的目标
    mask_without_small = morphology.remove_small_objects(mask,min_size=min_size,connectivity=2)
    # 连通域标记
    label_image = measure.label(mask_without_small,connectivity=2)
    #得到锚框
    regionprops= measure.regionprops(label_image)
    boundingbox = [region.bbox for region in regionprops]
    # minx,miny,maxx,maxy
    boundingbox = [Box(box[1],box[0],box[3],box[2],'capacitor') for box in boundingbox]
    return boundingbox

In [ ]:
def newgetboundingbox(imdata, min_size=2, ksize=(5,5)):
    # 通过膨胀去连通，可以min_size筛除小目标,或者通过锚框面积筛除
    # 通过kernel调整膨胀
    ## 膨胀
    if isinstance(imdata,Label):
        imdata=cv2.imread(imdata.label_path,cv2.IMREAD_GRAYSCALE)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, ksize=ksize)
    dilated = cv2.dilate(imdata.copy(), kernel, 1)
    # plt.imshow(dilated)
    bboxs = getboundingbox(dilated,min_size)
    # print(bboxs)
    ## 细化锚框
    tbboxs =getboundingbox(imdata,min_size)
    # print(tbboxs)
    cluster=[]
    for bbox in bboxs:
        tmp=[]
        for tbbox in tbboxs:
            if tbbox.xmin>=bbox.xmin and tbbox.ymin>=bbox.ymin and tbbox.xmax<=bbox.xmax and tbbox.ymax<=bbox.ymax:
                tmp.append(tbbox)
        cluster.append(tmp)

    res=[]
    for i in cluster:
        # print(i)
        xy=list(zip(*i))
        if xy:
            # minx,miny,maxx,maxy
            res.append(Box(min(xy[0]),min(xy[1]),max(xy[2]),max(xy[3]),'capacitor'))
    return res

In [ ]:
JPEG=Path(r'D:\Downloads\Capacitors\newvoc\train\images')
MASK=Path(r'D:\Downloads\Capacitors\newvoc\Segmentation') 
newgetboundingbox=partial(getboundingbox,min_size=500)
for im in JPEG.files():
    ma=MASK/f"{im.stem}.png"
    label=Label(im,ma,['capacitor'])
    label.convert(newgetboundingbox)
    # label.cv_show()
    label.unload('yolo')

imd=ImageDir(JPEG)
imd.create_yolo()
# 考虑DBSCAN是否可以实现